In [1]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install datasets
!pip install transformers[torch] -U

Looking in indexes: https://pypi.org/simple/


In [2]:
import os
import json
import numpy as np
import pandas as pd
import re
import string
from collections import Counter
from tqdm import tqdm

import torch
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
)
from datasets import load_dataset, Dataset, DatasetDict
from accelerate import Accelerator


/home/kar7mp5/Projects/2024-INHA-AI-CHALLENGE/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Validation / Test dataset

In [8]:
file_path = './content/train.csv'
train_data = pd.read_csv(file_path)

# 파일 경로에서 데이터를 읽어옴
file_path = './content/train.csv'
train_data = pd.read_csv(file_path)

# 데이터를 셔플하고 인덱스를 재설정
train_data = train_data.sample(frac=1).reset_index(drop=True)

# 검증 데이터와 학습 데이터로 분할
val_data = train_data[:10]
train_data = train_data[10:5000]

# 검증 데이터에서 질문과 답변 컬럼을 선택
val_label_df = val_data[['question', 'answer']]

# 학습 데이터를 datasets의 Dataset으로 변환
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_label_df)

# Model Load

In [3]:
# llama-2-ko-7b 모델 로드
model_id = "./models/20240704"
model = AutoModelForCausalLM.from_pretrained(model_id,
                                            torch_dtype="auto", load_in_4bit=True) # 한정된 메모리에서 사용하기 위해, 4비트 정밀도로 로드

tokenizer = AutoTokenizer.from_pretrained(model_id) # 본 모델의 토크나이저 로드
tokenizer.use_default_system_prompt = False  # 기본 시스템 프롬프트를 사용하지 않도록 설정

accelerator = Accelerator()

model, tokenizer = accelerator.prepare(model, tokenizer)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 6/6 [00:42<00:00,  7.00s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Define F1 score

In [4]:
def normalize_answer(s):
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text)
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)
        return text

    def white_space_fix(text):
        '''연속된 공백일 경우 하나의 공백으로 대체'''
        return ' '.join(text.split())

    def remove_punc(text):
        '''구두점 제거'''
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        '''소문자 전환'''
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()

    # 문자 단위로 f1-score를 계산 합니다.
    prediction_Char = []
    for tok in prediction_tokens:
        now = [a for a in tok]
        prediction_Char.extend(now)

    ground_truth_Char = []
    for tok in ground_truth_tokens:
        now = [a for a in tok]
        ground_truth_Char.extend(now)

    common = Counter(prediction_Char) & Counter(ground_truth_Char)
    num_same = sum(common.values())
    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(prediction_Char)
    recall = 1.0 * num_same / len(ground_truth_Char)
    f1 = (2 * precision * recall) / (precision + recall)

    return f1

def evaluate(ground_truth_df, predictions_df):
    predictions = dict(zip(predictions_df['question'], predictions_df['answer']))
    f1 = exact_match = total = 0

    for index, row in ground_truth_df.iterrows():
        question_text = row['question']
        ground_truths = row['answer']
        total += 1
        if question_text not in predictions:
            continue
        prediction = predictions[question_text]
        f1 = f1 + f1_score(prediction, ground_truths)

    f1 = 100.0 * f1 / total
    return {'f1': f1}


# Validation
- 로드한 모델에 대해 간단한 프롬프트를 입력하여 성능 검증

In [5]:
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [6]:
# 모델로 추론 후, 전처리를 수행한 뒤, 완성된 정답으로 반환합니다.
def generate_response(question_prompt):
    # 생성할 최대 토큰 수와, 답변 생성 수, 패딩 토큰의 idx를 지정하여 모델 파이프 라인을 설정하고, 답변을 생성합니다.
    response = qa_pipeline(question_prompt, max_new_tokens=70, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)[0]['generated_text']
    if "Answer:" in response:
            # Answer: 이후에 생성된 토큰 들만을 답변으로 사용합니다.
            response = response.split("Answer:", 1)[1][:20]

            # 토큰 반복 생성 및 노이즈 토큰 관련 처리
            if "Que" in response:
                response = response.split("Que", 1)[0]
            if "⊙" in response:
                response = response.split("⊙", 1)[0]
            if "Con" in response:
                response = response.split("Con", 1)[0]
    return response

In [9]:
#Google Colab T4 GPU 기준 10개 샘플 답안 생성에 1분 소요
predict_dict = {}
count = 0

for index, row in val_data.iterrows():
    try:
        context = row['context']
        question = row['question']


        if context is not None and question is not None:
            question_prompt = f"너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. \
                                Question에 대한 답변만 한 단어로만 최대한 간결하게 답변 해. \
                                Context: {context} Question: {question}\n Answer:"

            answer = generate_response(question_prompt)
            predict_dict[question] = answer
        else:
            predict_dict[question] = 'Invalid question or context'

        print("Answer for question:", question, ":", predict_dict[question])
        count += 1
        print("Processed count:", count)

    except Exception as e:
        print(f"Error processing question {e}")

Answer for question: 의왕시 화훼농가의 힘든 상황은 무엇 때문이야 :  코로나 19로 인한 화훼소비 감소와
Processed count: 1
Answer for question: 10일 오전 거래소를 기준으로 비트코인은 얼마에 거래됐어 :  7540만원
 
Processed count: 2
Answer for question: 위플이앤디가 12주 동안 HMR 제품을 협찬하기로 한 곳은 어디야 :  국방 FM '레이나의 건빵과 별사탕
Processed count: 3
Answer for question: 어느 기관에서 부동산 투기로 조직 개편에 들어섰어 :  한국토지주택공사(LH)이번에 새로 
Processed count: 4
Answer for question: 올해 1분기 전국 입주 예정 아파트는 몇 가구인지 :  8만387가구야. 
Processed count: 5
Answer for question: 해양수산부 장관이 나무 수종을 실시한 곳은 어디야 :  정부세종청사 5동 녹지공간
 
Processed count: 6
Answer for question: 이선홍 전주상공회의소 회장이 직무를 수행하는 동안 제일 만족스러웠던 일은 무엇이라고 했어 :  이선홍 회장은 전주상의 회관을 새로
Processed count: 7
Answer for question: GS25가 출시한 빵 브랜드 명칭은 무엇인가 :  브레디크
 
Processed count: 8
Answer for question: LG유플러스가 ESG위원회를 새로 설치하고 위원장 자리를 맡긴 사람이 누구야 :  제현주 사외이사야. 제현주 사외이사
Processed count: 9
Answer for question: 한국농어촌공사에서 9일 진행된 일은 뭐지 :  한국농어촌공사에서 9일 진행된 일은
Processed count: 10


In [10]:
val_inf_df = pd.DataFrame(list(predict_dict.items()), columns=['question', 'answer'])
val_inf_df.head()

question                answer
0              의왕시 화훼농가의 힘든 상황은 무엇 때문이야   코로나 19로 인한 화훼소비 감소와
1       10일 오전 거래소를 기준으로 비트코인은 얼마에 거래됐어             7540만원\n 
2  위플이앤디가 12주 동안 HMR 제품을 협찬하기로 한 곳은 어디야   국방 FM '레이나의 건빵과 별사탕
3           어느 기관에서 부동산 투기로 조직 개편에 들어섰어   한국토지주택공사(LH)이번에 새로 
4           올해 1분기 전국 입주 예정 아파트는 몇 가구인지            8만387가구야.

In [11]:
# f1-score 를 출력합니다.
results = evaluate(val_inf_df, val_label_df)
print(results)

{'f1': 68.68481305565705}


# Inference

In [12]:
file_path = './content/test.csv'
test_data = pd.read_csv(file_path) #, nrows = 10)

In [13]:
# 모델 추론
submission_dict = {}

for index, row in test_data.iterrows():
    try:
        context = row['context']
        question = row['question']
        id = row['id']

        if context is not None and question is not None:
            question_prompt = f"너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. \
                                Question에 대한 답변만 한 단어로만 최대한 간결하게 답변 해. \
                                Context: {context} Question: {question}\n Answer:"

            answer = generate_response(question_prompt)
            submission_dict[id] = answer
        else:
            submission_dict[id] = 'Invalid question or context'

    except Exception as e:
        print(f"Error processing question {e}")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


# Submission

In [14]:
# 제출
df = pd.DataFrame(list(submission_dict.items()), columns=['id', 'answer'])
df.to_csv( './content/submission.csv', index=False)